In [ ]:
import collections

import numpy as np
import pandas as pd
from inspyred.ec.analysis import fitness_statistics, generation_plot
from simanneal import Annealer
import math
from random import Random
from inspyred import ec #ec staat voor Evolutionary computation
from random import Random, randrange, random

In [ ]:
# Using simulated annealing
class RastriginProblem(Annealer): ##Een klasse die overerft van de Annealer klasse uit package simanneal
# 1. Maak een energyfunctie (objective function, optimalisatie-functie)
# De annaeler minimaliseert
 def energy(self): #Dit geeft terug hoe goed de oplossing is.
   s = self.state
   return -10 * len(s)+((s**2-10*np.cos(2*np.pi*s))).sum()

# 2. Bepaal de move functie. Waarbij je een nieuwe oplossing maakt die in de buurt ligt van de vorige oplossing.
 def move(self): #Implementatie van de move klasse
   # We wijzigen 1 van de variabelen uit de oplossings-array naar een nieuwe waarde
   s =self.state
   changing_index = np.random.randint(0,len(s)) #We geven gewoon één van de waarden van de solution een nieuwe waarde
   s[changing_index]+=np.random.normal(0,0.5,1)
   s = np.clip(s, a_min=-5.12, a_max=5.12)

In [ ]:
init_sol = np.random.uniform(-5.12,5.12, size=9)

In [ ]:

rastrigin=RastriginProblem(init_sol)
rastrigin.Tmax =100000
rastrigin.steps = 50000
rastrigin.updates = 100
rastrigin.Tmin = 100


solution, best = rastrigin.anneal()
print(solution)
best

# Genetic algorithms
## Rastrigin revisited
### 1. Maak een generate functie die een geldige oplossing kan maken

In [ ]:
#Geen initial solution nodig, wel een functie die een solution genereert. 
#De library gebruikt deze functie om een populatie te maken.
def generate(random = None, args =  None) -> []:
     return 

### 2. Maak een evaluate functie die een populatie van oplossingen evalueert

In [ ]:
#Evalueert voor een set van oplossingen (een populatie) de fitness.
# Je plaatst de ojective function best afzonderlijk en verwijst hiernaar in de evaluate functie
# De return is een array van oplossingen.
def evaluate(candidates, args = {}):
    return

#Equivalent van de energy functie. Bij inspyred kan je bij de uitvoering kiezen tussen minimalisatie of maximalisatie. Je moet de functie dus niet omkeren door er een min voor te zetten.

def obj_func(solution):
    return (10 * len(solution))+(solution**2-10*np.cos(2*np.pi*solution)).sum()


### 4. Genetisch algoritme configureren en uitvoeren

In [ ]:
#Imports
from inspyred import ec #ec staat voor Evolutionary computation
from random import Random
rand = Random()



# ga object maken
ga = ec.GA(rand)

# a. Heuristiek configureren
#------------------------------------------------
# observer, terminator, variator, replacer strategieën kiezen

# b. Heuristiek uitvoeren
#------------------------------------------------
# generate = , evaluate=: verwijs naar generate functie en evaluate functie
# selector: selectie strategie kiezen
# maximize: maximalisatie of minimalisatie
# bounder: de grenzen van de oplossingen
# strategie specieke parameters: zie uitleg van de gekozen strategieën
# eigen vastgelegde parameters: vb. num_inputs gebruikt in generate


## Traveling salesperson 
### Omzetten naar een probleem met binaire solutions

In [ ]:
distance_matrix = [0, 100, 125, 100, 75, 100, 0, 50, 75, 125, 125, 50, 0, 100, 125, 100, 75, 100, 0, 50, 75, 125, 125, 50, 0]

### Functies

In [ ]:
#Generate maakt een oplossing met 5 ééntjes
def generate(random = None, args =  None) -> []:
    size = args.get('num_cities',5)
    #Create a random but valid solution for the TSP problem
    solution = np.zeros(size*size,dtype=int)
    for i in range (0, size):
        j = np.random.randint(low=0, high=size)
        solution[i*size+j]=1
    return solution.tolist()

#Berekent de scores van een hele populatie
def evaluate(candidates, args = {}):
    weights = args.get('distance_matrix',[])
    fitness = []
    for candidate in candidates:
        fitness.append(obj_func(candidate,weights))
    return fitness

#Functie die op basis van de weights en de solution matrix een score geeft aan de oplossing.
def obj_func(solution, weights):
    n=int(math.sqrt(len(solution))) #Het aantal steden
    matrix_solution = np.array(solution).reshape(n,n)
    leaveOK=matrix_solution.sum(axis=0) #Tel in de matrix de som van de kolommen op
    arriveOK=matrix_solution.sum(axis=1) #Tel in de matrix de som van de rijen op
    notStayingOK = np.diag(matrix_solution).sum() #De som van diagonaal moet 0 zijn
    # #No subloops or infinite loop but one loop with length n
    loop_length = 0;
    city=0
    in_loop = True
    while(in_loop & (loop_length < n+1)):
        loop_length = loop_length + 1
        index = range(city*n,(city+1)*n, 1) # row of city
        next_city = 0
        while ((solution[index[next_city]] == 0) & (next_city < n-1)): next_city=next_city+1
        in_loop = (next_city != 0) & (solution[index[next_city]] == 1)
        city = next_city
    #Test if all of the conditions are fulfilled
    if (    (notStayingOK == 0) &
            (np.min(arriveOK) == 1) &
            (np.max(arriveOK) == 1) &
            (np.sum(arriveOK) == n) &
            (np.min(leaveOK) == 1) &
            (np.max(leaveOK) == 1) &
            (np.sum(leaveOK) == n) &
            (loop_length == n)          ):
        score=np.sum(np.multiply(solution, weights))  #value objective function
    else:
        score=10000*n #not a feasible solution, so very bad value for the objective function
    return score

### Configuratie van het genetisch algoritme en uitvoering

In [ ]:

from inspyred import ec #ec staat voor Evolutionary computation
from random import Random

rand = Random()
ga = ec.GA(rand)
ga.observer = ec.observers.plot_observer
ga.terminator = ec.terminators.evaluation_termination
ga.variator = [ec.variators.n_point_crossover, ec.variators.bit_flip_mutation]
ga.replacer = ec.replacers.plus_replacement #Bij de volgende populatie worden uit alle parents en offsprings de beste resultaten geselecteerd.
population = ga.evolve(
    generator=generate,
    evaluator=evaluate,
    selector = ec.selectors.fitness_proportionate_selection, # dit is het rouletteselectiemechanisme (wie mag paren?
    pop_size=1000,
    maximize= False,
    #bounder=ec.DiscreteBounder([0,1]), #bounder die enkel 0 en 1 oplevert (is in principe niet nodig in dit geval)
    max_evaluations=50000,
    mutation_rate=0.01,
    num_crossover_points = 3,
    distance_matrix=distance_matrix
    )
print(population[0])
ec.analysis.fitness_statistics(population)

# Oefeningen

## Gutter

### Simmulated annealing

In [ ]:
class VolumeProblem(Annealer):
    def energy(self):
        s = self.state
        return -s*(1-s/2)

    def move(self):
        s = self.state
        s += np.random.normal(0, 0.1, 1)

init_sol = np.random.uniform(0.1, 1, size=1)
rastrigin = VolumeProblem(init_sol)
rastrigin.Tmax = 100000
rastrigin.steps = 1000
rastrigin.updates = 5000
rastrigin.Tmin = 100

solution, best = rastrigin.anneal()
print(solution)
print(best)

In [ ]:
class GutterProblem(Annealer):
    def energy(self):
        b = self.state[0]
        h = (1 - b)/2
        return -b*h

    def move(self):
        self.state += np.random.normal(0,0.1,1)
        return

init_sol = [0]

gutter=GutterProblem(init_sol)
gutter.Tmax =100000
gutter.steps = 1000
gutter.updates = 5000
gutter.Tmin = 100

#Doelfunctie resultaat van de initiële oplussing
gutter.energy()
gutter.anneal()
#initial solution


### Genetic algorithms

## KNAPSACK

### Simmulated annealing

In [ ]:
df = pd.read_csv('../datasets/Knapsack Items.csv', index_col=0)
weights = df['gewichten(gr)']
values = df['waarde']

class KnapsackProblem(Annealer):
    def energy(self):
        b = self.state
        total_weight = (b * weights).sum()
        total_values = (b * values).sum()
        if total_weight > 750:
            return -(750 - total_weight)
        else:
            return -total_values

    def move(self):
        index= self.state[np.random.randint(0,len(values))]
        if (self.state[index]==0):
            self.state[index]=1
        else:
            self.state[index]=0
        return
    
init_sol = np.random.choice([0,1],len(values)) #initial solution
gutter=KnapsackProblem(init_sol)
gutter.Tmax =100000
gutter.steps = 10000
gutter.updates = 5000
gutter.Tmin = 100


gutter.anneal()


### Genetic algorithms

#### Vervolledig de generate en evaluate functies

In [ ]:
def obj_funct(weights, values, selection):
    total_weight = (selection* weights).sum()
    total_values = (selection* values).sum()
    if total_weight > 750:
        return total_values/3
    else:
        return total_values

def evaluate(candidates, args = {}):

    return 

def generate(random = None, args = {}) -> []:
    return 

In [ ]:
df = pd.read_csv('../datasets/Knapsack Items.csv', index_col=0)
weights = df['gewichten(gr)']
values = df['waarde']

### Configuratie van het genetisch algoritme en uitvoering
Gebruik n_point_crossover en bit_flip_mutation als variators, gebruik de descrete bounder en de tournament selection.